# Iskanje in prenos posnetkov

## Viri satelitskih posnetkov

## Načini dostopa

## Potrebne knjižnice

Satelitski posnetki so običajne rastrske datoteke, torej (velike) matrike. Za njihovo branje, obdelavo in pretvorbo, potrebujemo več knjižnic, med katerimi so najpomembnejše:

* NumPy
* Sentinel Hub

In [ ]:
# Potrebne knjižnice
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sentinelhub as shub

In [ ]:
# Paremetri beležnice
%reload_ext autoreload
%autoreload 2
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]

## Sentinel Hub račun

Za dostop do Sentinel Huba je potrebna registracija. Ki imamo uporabniško ime in geslo, moramo na konfikuratorju ustvariti sloje oziroma storitve do katerih želmo dostopati. Konfigurator je dostopen na naslovu:  
https://apps.sentinel-hub.com/configurator/#/

![](./slike/sh_configuration_utility.png)

Za dostop prek Pythona potrebujemo ID ustvarjene konfiguracije. ID ključ je vezan na osebo. Praviloma ga ne delimo in zato tudi ne zapisujemo beležnico. Lahko ga preberemo iz datoteke, v skrajnem primeru definiramo spremenljivko `INSTANCE_ID`, ki je niz znakov.

Dostop do posameznih slojev je mogoč prek njihovega ID.

![](./slike/sh_configuration_utility_layers.png)

In [ ]:
# ID ključ mora biti definiran, preberemo ga iz datoteke
sentinelhub_id = 'sentinelhub.id'
f = open(sentinelhub_id,'r')
exec(f.readline())

In [ ]:
if 'INSTANCE_ID' in locals():
    print("Sentinel Hub INSTANCE_ID obstaja")
    print(INSTANCE_ID)

## Iskanje satelitskih posnetkov

Poiskali bomo sateliske posnetke območja med Kranjem in Škofjo Loko. Najprej definiramo območje. Sentinel Hub pozna različne koordinatne sisteme, mi bomo uporabili kar WGS84 in torej geografske koordinate. Potrebujemo geografsko širini in dolžino zgornjega levega in spodnjega desnega kota. Oboje bi lahko prebrali iz ustrezne datoteke, recimo GeoJSON ali SHP. Koordinate lahko odčitamo iz poljubne spletne karte ([Google Maps](https://www.google.si/maps/), [GeoJSON.io](http://geojson.io/)) ali namiznega GIS programa (QGIS, ArcMap). Ustvariti moramo omočje `sentinelhub.common.BBox` v ustreznem koordinatnnem sistemu `sentinelhub.common.CRS`.

![](./slike/geojson_io.jpg)

In [ ]:
kranj_wgs84 = [14.26, 46.26, 14.51, 46.11]
kranj_bbox = shub.BBox(bbox=kranj_wgs84, crs=shub.CRS.WGS84)

Preverimo, če je območje pravilno delfiniramo in če se lahko povežemo s Sentinel Hubom. Najprej bomo do servisa dostopali prek lahko protokola WMS.

In [ ]:
# Datum posnetka za malo oblaki
kranj_date = '2017-07-20'

In [ ]:
wms_true_color_request = shub.WmsRequest(layer='TRUE-COLOR-S2-L1C',
                                    bbox=kranj_bbox,
                                    time=kranj_date,
                                    width=1024, height=1024,
                                    instance_id=INSTANCE_ID)

In [ ]:
wms_true_color_img = wms_true_color_request.get_data()

In [ ]:
# Kanali v WMS so BGR, Python ima RGB
plt.imshow(wms_true_color_img[-1][..., [2, 1, 0]])

In [ ]:
print('Sentinel-2: ', wms_true_color_request.get_dates()[-1])

## Iskanje posentkov s parametri

In [ ]:
search_bbox = shub.BBox(bbox=kranj_bbox, crs=shub.CRS.WGS84)
search_time_interval = ('2015-01-01', '2018-12-31')

In [ ]:
# Poiščemo posnetke
wfs_iterator = shub.WebFeatureService(search_bbox, search_time_interval,
                                      data_source=shub.DataSource.SENTINEL2_L1C,
                                      maxcc=1.0, instance_id=INSTANCE_ID)

In [ ]:
for tile_info in wfs_iterator:
    print(tile_info)

In [ ]:
# Vzamemo samo properties in naredimo pandas DF
shub_wfs_df = pd.DataFrame(wfs_iterator.tile_list)['properties'].apply(pd.Series)
# Stolpec date je datum
shub_wfs_df['date'] = pd.to_datetime(shub_wfs_df['date'])

In [ ]:
shub_wfs_df.head()

In [ ]:
shub_wfs_df.describe()

In [ ]:
shub_wfs_df.plot(x='date', y='cloudCoverPercentage', style='o-')

## Poiščemo posnetke z malo oblakov

In [ ]:
shub_wfs_df_cc = shub_wfs_df[shub_wfs_df["cloudCoverPercentage"] < 10]

In [ ]:
len(shub_wfs_df_cc)

In [ ]:
shub_wfs_df_cc

Izberemo zanimive posnetke, recimo enega v posameznem letu, recimo posnetke:  
2015-07-11
2016-08-14
2017-07-20  

In [ ]:
# Definiramo datume
kranj_dates_aug = ["2015-07-11", "2016-08-14", "2017-07-20"]
# Izberemo samo vrstice s temi datumi
shub_wfs_df_sel = shub_wfs_df_cc[shub_wfs_df_cc['date'].isin(kranj_dates_aug)]

In [ ]:
shub_wfs_df_sel

## Zanimivi podatki

In [ ]:
# Prenesemo kanala 4 in 8, ki ju rabimo za izračun NDVI
bands = ['B08', 'B04']
# Metapodatki
metafiles = ['tileInfo', 'preview', 'qi/MSK_CLOUDS_B00']
# Mapa z datotekami
data_folder = './posnetki'

## Prikaz posnetka

In [ ]:
kranj_prvi_id = shub_wfs_df_sel["id"].iloc[0]

In [ ]:
tile_name, time, aws_index = shub.AwsTile.tile_id_to_tile(kranj_prvi_id)
request = shub.AwsTileRequest(tile=tile_name, time=time, aws_index=aws_index,
                              bands=bands, metafiles=metafiles, data_folder=data_folder)

In [ ]:
data_list = request.get_data()

In [ ]:
b8a, b10, tile_info, preview, cloud_mask = data_list

In [ ]:
# 8 kanal
plt.imshow(b8a, cmap='gray')

In [ ]:
plt.imshow(preview)

## Prenos podatkov

Prenesemo vse tri posnetke.

In [ ]:
for index, row in shub_wfs_df_sel.iterrows():
    print('Sentinel-2 ', row["date"].strftime('%Y-%m-%d'), row["id"])

In [ ]:
for index, row in shub_wfs_df_sel.iterrows():
    print('Prenašam ', row["date"].strftime('%Y-%m-%d'))
    tile_id = row["id"]
    tile_name, time, aws_index = shub.AwsTile.tile_id_to_tile(tile_id)
    print("Tile ID: ", tile_id)
    print("Tile: ", tile_name)
    request = shub.AwsTileRequest(tile=tile_name, time=time, aws_index=aws_index,
                                  bands=bands, metafiles=metafiles, data_folder=data_folder)
    request.save_data()